<a href="https://colab.research.google.com/github/mtzig/badwriting_OCR/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/mtzig/badwriting_OCR.git
%cd /content/badwriting_OCR/

Cloning into 'badwriting_OCR'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 309 (delta 40), reused 96 (delta 30), pack-reused 192
Receiving objects: 100% (309/309), 8.25 MiB | 13.10 MiB/s, done.
Resolving deltas: 100% (85/85), done.
/content/badwriting_OCR


In [3]:
!pip install -q transformers
!pip install -q datasets jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.4 MB/s eta 0:00:00


In [4]:
from utils import model_utils
from transformers import AdamW
from tqdm.notebook import tqdm
import torch
from PIL import Image

/content/badwriting_OCR/utils/model_utils.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
train_dataloader,eval_dataloader = model_utils.get_dataloaders(dataset_type='t')
model = model_utils.getModel(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrOCRProcessor
import pandas as pd

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")

def compute_outputs(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)


    return pred_str, label_str

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [13]:
base_preds, base_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    base_preds.extend(p)
    base_labels.extend(l)

base_df = pd.DataFrame(data={'preds':base_preds, 'lables':base_labels})
base_df.to_csv('base.csv', index=False)




  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [14]:
full_train_cer = []

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = model_utils.compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer

   full_train_cer.append(valid_cer / len(eval_dataloader))

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 0: 8.78665033976237


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

Validation CER: 1.0


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 9: 3.805255969365438


  0%|          | 0/4 [00:00<?, ?it/s]

Validation CER: 1.0866511968734733


In [15]:
full_preds, full_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    full_preds.extend(p)
    full_labels.extend(l)

full_df = pd.DataFrame(data={'preds':full_preds, 'lables':full_labels})
full_df.to_csv('full.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
model = model_utils.getModel(device)
# for pn, p in model.named_parameters():
#   if pn != 'decoder.output_projection.weight':
#     p.requires_grad = False

for pn, p in model.named_parameters():
  if pn.startswith('encoder'):
    p.requires_grad = False

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
freeze_train_cer = []

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = model_utils.compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer

   freeze_train_cer.append(valid_cer / len(eval_dataloader))

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 0: 9.190086285273233


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

Validation CER: 0.8036286164034087


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 9: 0.8716660688320795


  0%|          | 0/4 [00:00<?, ?it/s]

Validation CER: 0.792392661347229


In [18]:
freeze_preds, freeze_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    freeze_preds.extend(p)
    freeze_labels.extend(l)

freeze_df = pd.DataFrame(data={'preds':freeze_preds, 'lables':freeze_labels})
freeze_df.to_csv('freeze.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [19]:
cer_df = pd.DataFrame(data={'full':full_train_cer, 'freeze':freeze_train_cer})
cer_df.to_csv('cer.csv', index=False)